In [1]:
# aircraft_rul_model.py

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import os

# ========== STEP 1: LOAD AND PREPROCESS DATA ==========
# Download the CMAPSS FD001 dataset and load it
# https://www.nasa.gov/content/prognostics-center-of-excellence-data-set-repository
df = pd.read_csv("/content/train_FD001.txt", sep=' ', header=None)
df.dropna(axis=1, inplace=True)

# Set column names
cols = ['unit_number', 'time_in_cycles',
        'operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + \
       [f'sensor_{i}' for i in range(1, 22)]
df.columns = cols

# Drop unused sensors
drop_sensors = ['sensor_1', 'sensor_5', 'sensor_10']
df.drop(columns=drop_sensors, inplace=True)

# ========== STEP 2: COMPUTE RUL ==========
rul_df = df.groupby('unit_number')['time_in_cycles'].max().reset_index()
rul_df.columns = ['unit_number', 'max_cycle']
df = df.merge(rul_df, on='unit_number', how='left')
df['RUL'] = df['max_cycle'] - df['time_in_cycles']
df.drop(columns='max_cycle', inplace=True)

# ========== STEP 3: PREPARE FEATURES ==========
feature_cols = [col for col in df.columns if col not in ['unit_number', 'time_in_cycles', 'RUL']]
X = df[feature_cols]
y = df['RUL']

# ========== STEP 4: SCALING AND SPLITTING ==========
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ========== STEP 5: MODEL TRAINING ==========
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ========== STEP 6: EVALUATION ==========
y_pred = model.predict(X_test)
print("📊 Model Performance on Test Set:")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")

# ========== STEP 7: SAVE MODEL AND SCALER ==========
os.makedirs("model", exist_ok=True)
pickle.dump(model, open("model/rul_model.pkl", "wb"))
pickle.dump(scaler, open("model/scaler.pkl", "wb"))
pickle.dump(feature_cols, open("model/feature_list.pkl", "wb"))

# ========== STEP 8: PREDICT RUL FOR NEW ENGINE ==========
# Example sensor input (replace these with real sensor values)
input_data = {
    'operational_setting_1': 0.0,
    'operational_setting_2': 0.0,
    'operational_setting_3': 100.0,
    'sensor_2': 642.82,
    'sensor_3': 1589.66,
    'sensor_4': 1405.6,
    'sensor_6': 21.61,
    'sensor_7': 39.06,
    'sensor_8': 23.28,
    'sensor_9': 100.0,
    'sensor_11': 39.0,
    'sensor_12': 23.23,
    'sensor_13': 47.47,
    'sensor_14': 522.49,
    'sensor_15': 2388.06,
    'sensor_16': 8138.48,
    'sensor_17': 8.42,
    'sensor_18': 0.03,
    'sensor_19': 391.0,
    'sensor_20': 2388.0,
    'sensor_21': 100.0
}

# Load model, scaler, and feature list
model = pickle.load(open("model/rul_model.pkl", "rb"))
scaler = pickle.load(open("model/scaler.pkl", "rb"))
feature_list = pickle.load(open("model/feature_list.pkl", "rb"))

# Convert input into DataFrame and reorder
X_input_df = pd.DataFrame([input_data])[feature_list]

# Scale and predict
X_scaled_input = scaler.transform(X_input_df)
predicted_rul = model.predict(X_scaled_input)[0]

print(f"\n🔧 Predicted Remaining Useful Life (RUL): {predicted_rul:.2f} cycles")

📊 Model Performance on Test Set:
MAE: 29.64
MSE: 1717.62
R² Score: 0.6241

🔧 Predicted Remaining Useful Life (RUL): 201.62 cycles
